In [1]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta
import requests

In [2]:
def fetch_candles(pair: str, before: int, after: int, periods: int, save: bool = False) -> pd.DataFrame:
    '''
    exchange: ftx
    pair: fttusd-perpetual-futures
    '''
    url = f"https://api.cryptowat.ch/markets/ftx/{pair}/ohlc"
    query = {'before': before, 'after': after, 'periods': periods}
    candles = requests.get(url, params=query).json()["result"][f"{periods}"]

    t, o, h, l, c, v = [], [], [], [], [], []
    for candle in candles:
        t.append(datetime.fromtimestamp(candle[0]))
        o.append(candle[1])
        h.append(candle[2])
        l.append(candle[3])
        c.append(candle[4])
        v.append(candle[5])

    candles = pd.DataFrame({"Date": t, "Open": o, "High": h, "Low": l, "Close": c, "Volume": v}).set_index("Date")

    if save:
        pth = f"../data/{periods}/{pair}/"
        os.makedirs(pth, exist_ok=True)
        candles.to_csv(pth + f"{candles.index[0].strftime('%Y-%m-%d')}.csv")

    return candles

In [3]:
pair = "btcusd-perpetual-futures"
current_time = datetime.fromtimestamp(int(time.time()))
after = current_time - timedelta(days=300)
print(current_time)
print(after)
before = int(datetime.timestamp(current_time))
after = int(datetime.timestamp(after))
periods = 60 * 15
print(before, after)
print(before, after + 60 * 60 * 24)

candles = fetch_candles(pair, before, after, periods)
print(candles)

2021-07-24 14:47:35
2020-09-27 14:47:35
1627105655 1601185655
1627105655 1601272055
                      Open   High    Low  Close    Volume
Date                                                     
2021-05-23 02:45:00  38225  38326  37818  37911  948.7391
2021-05-23 03:00:00  37919  38149  37843  38010  403.8273
2021-05-23 03:15:00  38008  38166  37687  37961  658.3518
2021-05-23 03:30:00  37969  37976  37652  37677  353.5720
2021-05-23 03:45:00  37677  37810  37340  37418  656.4611
...                    ...    ...    ...    ...       ...
2021-07-24 13:45:00  33685  33776  33668  33772  318.9773
2021-07-24 14:00:00  33772  33873  33700  33707  682.6960
2021-07-24 14:15:00  33707  33733  33584  33644  576.9839
2021-07-24 14:30:00  33644  33686  33585  33672  408.7255
2021-07-24 14:45:00  33672  33721  33639  33711  355.9761

[5999 rows x 5 columns]


In [11]:
pairs = ["btcusd", "ethusd"]
url = "https://api.cryptowat.ch/markets/ftx"
markets = requests.get(url).json()["result"]
for market in markets:
    if market["pair"] in pairs:
        print(market)

{'id': 62576, 'exchange': 'ftx', 'pair': 'btcusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/ftx/btcusd'}
{'id': 62579, 'exchange': 'ftx', 'pair': 'ethusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/ftx/ethusd'}


In [ ]:
start = int(datetime.timestamp(datetime(2021, 1, 1)))
end = int(datetime.timestamp(datetime(2021, 6, 30)))
for pair in pairs:
    fetch_candles(pair, end, start, 60 * 15, save=True)